In [1]:
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [5]:
%ls 

layer1.json          preprossData.csv  RecipeGenerator/
LICENSE              README.md         RecipeGenerator.xcodeproj/
Preprocessing.ipynb  Recipe/           RecipeTraining.ipynb


In [4]:
%cd gdrive/MyDrive/RecipeGen/RecipeGeneratorNLP/

/content/gdrive/MyDrive/RecipeGen/RecipeGeneratorNLP


In [6]:
df = pd.read_csv('preprossData.csv')

In [7]:
df.head()

,Unnamed: 0,title,ingred,instructions
0,0,Worlds Best Mac and Cheese,"penne|beechers flagship cheese sauce|cheddar, ...",preheat the oven to 350 f. butter or oil an 8-...
1,1,Dilly Macaroni Salad Recipe,c. eow macaroni|c. cubed american cheese|c. sl...,cook macaroni according to package directions;...
2,2,Gazpacho,"maes, quartered|kosher salt|red onion, cut in ...",add the tomatoes to a food processor with a pi...
3,3,Crunchy Onion Potato Bake,"milk|water|butter|mashed potaes, box, homesty...",preheat oven to 350 degrees fahrenheit.spray p...
4,4,Cool 'n Easy Creamy Watermelon Pie,package watermelon gelatin|boiling water|packa...,dissolve jello in boiling water.allow to cool ...


In [8]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [9]:
from transformers import AdamW, T5ForConditionalGeneration, T5Tokenizer, get_linear_schedule_with_warmup
import torch
from torch.utils.data import Dataset, DataLoader
# import pytorch_lighting as pl
from sklearn.model_selection import train_test_split
import textwrap

In [10]:
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [11]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("google/t5-v1_1-base")

model = AutoModelForSeq2SeqLM.from_pretrained("google/t5-v1_1-base")

In [26]:
sample = tokenizer(df['ingred'][0], df['instructions'][0])
input_id = sample['input_ids']


In [27]:
tokenizer.decode(sample['input_ids'])

'penne|beechers flagship cheese sauce|cheddar, grated|gruyere cheese, grated|chipotle chili powder|unsalted butter|all-purpose flour|milk|semihard cheese, grated|semisoft cheese, grated|kosher salt|chipotle chili powder|garlic powder</s> preheat the oven to 350 f. butter or oil an 8-inch baking dish.cook the penne 2 minutes less than package directions.(it will finish cooking in the oven.)rinse the pasta in cold water and set aside.combine the cooked pasta and the sauce in a medium bowl and mix carefully but thoroughly.scrape the pasta into the prepared baking dish.sprinkle the top with the cheeses and then the chili powder.bake, uncovered, for 20 minutes.let the mac and cheese sit for 5 minutes before serving.melt the butter in a heavy-bottomed saucepan over medium heat and whisk in the flour.continue whisking and cooking for 2 minutes.slowly add the milk, whisking constantly.cook until the sauce thickens, about 10 minutes, stirring frequently.remove from the heat.add the cheeses, sal

In [28]:
preds = [ tokenizer.decode(input_id, skip_special_token = True, clean_up_tokenization_spaces=True) for input_id in sample['input_ids'] ]

In [29]:
" ".join(preds)

' p enne | be e cher s flagship cheese sauce | ched dar , gr ated | gru yer e cheese , gr ated | chi pot le chili powder | un s al ted butter | all - purpose flour | milk | s e m i hard cheese , gr ated | s emis of t cheese , gr ated | kos her salt | chi pot le chili powder | gar lic powder </s>  p reheat the oven to 350  f . butter or oil an 8 - inch baking dish . co ok the  p enne 2 minutes less than package directions . ( it will finish cooking in the oven . ) r in s e the pasta in cold water and set aside . com bine the cooked pasta and the sauce in  a medium bowl and mix carefully but thoroughly . s cra pe the pasta into the prepared baking dish . s prin kle the top with the cheese s and then the chili powder . bak e ,  uncovered , for 20 minutes . let the mac and cheese sit for 5 minutes before serving . mel t the butter in  a heavy - bottom e d saucepan over medium heat and whisk in the flour . continu e whisk ing and cooking for 2 minutes . s low ly add the milk , whisk ing con